In [ ]:
import numpy as np
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
import tensorflow as tf
from matplotlib import pyplot as plt
import laspy as lp
import matplotlib
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree

# 1. Load the data

In [2]:
X_train = np.load('Final_train_data.npy')
y_train = np.load('Final_targets.npy')
X_test = np.load('Final_validation_data.npy')
y_test = np.load('Final_validation_targets.npy')

# 2. Find the neighbours for each point

In [3]:
# Define the number of nearest neighbors to consider
k_neighbors = 10

# Calculate the nearest neighbors for each data point using KD-Tree
tree = KDTree(X_train)
tree2 = KDTree(X_test)
nearest_neighbors_train = tree.query(X_train, k=k_neighbors+1, return_distance=False)[:, 1:]
nearest_neighbors_test = tree2.query(X_test, k=k_neighbors+1, return_distance=False)[:, 1:]

# 3. Prepare the data format for the CNN models

In [4]:
# Generate the input data for training the model
def generate_input_array(X, nearest_neighbors):
    input_array = np.concatenate([X[:,3:][nearest_neighbors[:, i]] for i in range(k_neighbors)], axis=1)
    return input_array.reshape(-1, k_neighbors, X[:,3:].shape[1], 1)

input_array_train = generate_input_array(X_train, nearest_neighbors_train)
input_array_test = generate_input_array(X_test, nearest_neighbors_test)

# Normalize the nearest_neighbors arrays
scaler = StandardScaler()
input_array_train = scaler.fit_transform(input_array_train.reshape(input_array_train.shape[0], -1))
input_array_test = scaler.transform(input_array_test.reshape(input_array_test.shape[0], -1))
input_array_train = input_array_train.reshape(-1, k_neighbors, X_train[:,3:].shape[1], 1)
input_array_test = input_array_test.reshape(-1, k_neighbors, X_test[:,3:].shape[1], 1)


# First CNN Model

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(128, kernel_size=3, activation='relu', input_shape=(k_neighbors, 8, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

model.summary()

# Train and evaluate the model

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(input_array_train, y_train, epochs=8, batch_size=64)

loss, accuracy = model.evaluate(input_array_test, y_test)

# Print the accuracy as a percentage
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Predict the first model

In [ ]:
prediction = model.predict(input_array_test)
y_pred = np.argmax(prediction, axis=-1)
y_pred

# Second CNN Model

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same', input_shape=(k_neighbors, 8)))
model.add(BatchNormalization())
model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(64, kernel_size=2, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(64, kernel_size=2, activation='relu', padding='same'))
model.add(Dropout(0.3))

model.add(Conv1D(32, kernel_size=2, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(32, kernel_size=2, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(7, activation='softmax'))

model.summary()


# Train and evaluate the model

In [ ]:

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(input_array_train, y_train, epochs=8, batch_size=64)

loss, accuracy = model.evaluate(input_array_test, y_test)

# Print the accuracy as a percentage
print("Accuracy: {:.2f}%".format(accuracy * 100))


# Predict Second model

In [ ]:
y_pred_modified = np.argmax(model.predict(input_array_test), axis=-1)

# Per class accuraccy 

In [ ]:
cm_modified = confusion_matrix(y_test, y_pred_modified)
class_accuracies_modified = cm_modified.diagonal() / cm_modified.sum(axis=1)
for i, acc in enumerate(class_accuracies_modified):
    print(f"Accuracy for class {i} (Modified architecture): {acc:.2f}")

# Save output as las file

In [15]:
print_coord = x_new_test2[:, :3]
fn = "CNN_chat4.las"

vals = np.linspace(0, 1, 100)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20(vals))
header = lp.header.Header()
header.data_format_id = 2
fp = lp.file.File(fn, mode = 'w', header = header)
fp.header.scale = [0.01, 0.01, 0.01]
fp.header.offset = [min(print_coord[:,0]), min(print_coord[:,1]), min(print_coord[:,2])]
fp.x = print_coord[:, 0]
fp.y = print_coord[:, 1]
fp.z = print_coord[:, 2]
fp.pt_src_id = y_pred
#fp.intensity = intensity
fp.close()